## Saving 1 frame.


In [2]:
import cv2
import os

# 1. Define paths
video_path = '/media/holidayj/Documents/Videos/videos/platform/euljiro/euljiro_20221101_17_20.mp4'
output_folder = 'output_frames'  # The folder to be created
output_filename = 'euljiro_capture.jpg'

# 2. Create the output folder if it doesn't exist
# exist_ok=True prevents errors if the folder already exists
os.makedirs(output_folder, exist_ok=True)
print(f"Folder '{output_folder}' is ready.")

# 3. Load the video
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: Could not open video at {video_path}")
else:
    # 4. Read the first frame
    ret, frame = cap.read()

    if ret:
        # 5. Construct the full save path
        full_save_path = os.path.join(output_folder, output_filename)
        
        # 6. Save the frame
        cv2.imwrite(full_save_path, frame)
        print(f"Success: Image saved to: {full_save_path}")
    else:
        print("Error: Video opened, but could not read the first frame.")

# 7. Cleanup
cap.release()

Folder 'output_frames' is ready.
Success: Image saved to: output_frames/euljiro_capture.jpg


## Setting position for 320*320

In [ ]:
import cv2
import os

# --- CONFIGURATION ---
video_path = '/media/holidayj/Documents/Videos/videos/platform/euljiro/euljiro_20221101_17_20.mp4'
output_folder = 'output_frames'
crop_size = 320

# "Just a little far from the corner"
# Set these to 0 for the absolute corner. Increase them to move the box inward.
margin_top   = 0    # Move down 50 pixels from the top edge
margin_right = 120  # Move left 100 pixels from the right edge
# ---------------------

os.makedirs(output_folder, exist_ok=True)

cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()

if ret:
    # 1. Get image dimensions automatically (usually 1920 x 1080 for HD)
    height, width, _ = frame.shape
    
    # 2. Calculate the exact coordinates
    # Y-coordinates (Vertical): Top starts at 0 + margin
    y_start = margin_top
    y_end = y_start + crop_size

    # X-coordinates (Horizontal): Right side is 'width', so we subtract from there
    x_end = width - margin_right
    x_start = x_end - crop_size

    print(f"Original Resolution: {width}x{height}")
    print(f"Cropping Area -> X: {x_start} to {x_end}, Y: {y_start} to {y_end}")

    # 3. Perform the Crop
    # NOTE: Numpy uses [y, x] formatting!
    cropped_frame = frame[y_start:y_end, x_start:x_end]

    # 4. Save
    full_save_path = os.path.join(output_folder, 'cropped_320x320.jpg')
    cv2.imwrite(full_save_path, cropped_frame)
    print(f"Saved cropped image to {full_save_path}")

    # Optional: Show the result to verify
    # cv2.imshow("Original (Marked)", cv2.rectangle(frame, (x_start, y_start), (x_end, y_end), (0, 255, 0), 2))
    # cv2.imshow("Cropped", cropped_frame)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

cap.release()


Original Resolution: 1920x1080
Cropping Area -> X: 1480 to 1800, Y: 0 to 320
Saved cropped image to output_frames/cropped_320x320.jpg


## Extracting frames

In [7]:
import cv2
import os
import math
from multiprocessing import Pool, cpu_count

# --- CONFIGURATION ---
VIDEO_PATH = '/media/holidayj/Documents/Videos/videos/platform/euljiro/euljiro_20221101_17_20.mp4'
OUTPUT_FOLDER = 'extraction'
INTERVAL_SEC = 2
CROP_SIZE = 320
MARGIN_RIGHT = 120
MARGIN_TOP = 0
# ---------------------

def extract_frames_worker(args):
    """
    Worker function to be run by each CPU core.
    args contains: (video_path, list_of_frame_indices_to_capture, output_dir, crop_params)
    """
    video_path, frames_to_process, output_dir, (x_start, y_start) = args
    
    # Each process opens its own handle to the video file
    cap = cv2.VideoCapture(video_path)
    
    for frame_idx in frames_to_process:
        # Seek to specific frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        
        if ret:
            # Crop: frame[y:y+h, x:x+w]
            cropped = frame[y_start : y_start + CROP_SIZE, 
                            x_start : x_start + CROP_SIZE]
            
            # Generate consistent filename with zero-padding (6 digits)
            # e.g., euljiro_frame_000060.jpg
            filename = f"euljiro_frame_{frame_idx:06d}.jpg"
            save_path = os.path.join(output_dir, filename)
            
            cv2.imwrite(save_path, cropped)
    
    cap.release()
    return len(frames_to_process)

def main():
    # 1. Setup Folders
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)

    # 2. Analyze Video Metadata
    cap = cv2.VideoCapture(VIDEO_PATH)
    if not cap.isOpened():
        print("Error: Cannot open video.")
        return

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    cap.release() # Release main handle, workers will open their own

    # 3. Calculate Crop Coordinates
    # X: Width - Margin_Right - Crop_Size
    x_start = width - MARGIN_RIGHT - CROP_SIZE
    y_start = MARGIN_TOP
    
    print(f"Video FPS: {fps}")
    print(f"Total Frames: {total_frames}")
    print(f"Crop X: {x_start} to {x_start + CROP_SIZE}")
    print(f"Crop Y: {y_start} to {y_start + CROP_SIZE}")

    # 4. specific the target frames (Every 2 seconds)
    frame_step = int(fps * INTERVAL_SEC)
    target_indices = list(range(0, total_frames, frame_step))
    
    print(f"Extracting {len(target_indices)} frames using {cpu_count()} CPUs...")

    # 5. Distribute work across CPUs
    num_cpus = cpu_count()
    chunk_size = math.ceil(len(target_indices) / num_cpus)
    
    # Create chunks of tasks
    tasks = []
    for i in range(0, len(target_indices), chunk_size):
        chunk = target_indices[i : i + chunk_size]
        # Pack arguments: path, specific frames, folder, crop coordinates
        tasks.append((VIDEO_PATH, chunk, OUTPUT_FOLDER, (x_start, y_start)))

    # 6. Execute in Parallel
    with Pool(processes=num_cpus) as pool:
        pool.map(extract_frames_worker, tasks)

    print("Done! Frames saved in:", os.path.abspath(OUTPUT_FOLDER))

if __name__ == '__main__':
    main()

Video FPS: 29.999683531044475
Total Frames: 331751
Crop X: 1480 to 1800
Crop Y: 0 to 320
Extracting 5623 frames using 8 CPUs...
Done! Frames saved in: /media/holidayj/Documents/github/dup/annotation/extraction
